In [31]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import sympy
from sympy import *
import itertools
from IPython.display import display
init_printing() #allows nice math displays
from qiskit_ionq import IonQProvider, GPIGate, GPI2Gate, MSGate
from qiskit.providers.jobstatus import JobStatus
from qiskit import QuantumCircuit

In [2]:
provider = IonQProvider("4sqiVkjcPWcvSxBpFmolrqPuGtMoGh7S")

ionq_simulator_backend = provider.get_backend("ionq_simulator", gateset = "native")
ionq_qpu_backend = provider.get_backend("ionq_qpu")

In this notebook, I will use the QuantumMatrix class provided in this link: https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/terra/qis_adv/Clifford_Group.ipynb. It offers a good way to visualize the matrix and also takes care of the phase and coefficients thus allowing more direct comparison

In [3]:
class QuantumMatrix():
    def __init__(self, m, coeff = 1):
        self.matrix = sympy.Matrix(m)
        self.coefficient = coeff
        self.canonize()
        
    def canonize(self):
        a = next((x for x in self.matrix if x != 0), None)
        if a is not None: #zero vector
            for i,j in itertools.product([0,1], [0,1]):
                self.matrix[i,j] = sympy.simplify(self.matrix[i,j] / a)
        self.coefficient = sympy.simplify(self.coefficient * a) 
            
    def __str__(self):
        coeff_string = ""
        if self.coefficient != 1:
            coeff_string = "{} * ".format(self.coefficient)
        return "{}[[{}, {}], [{}, {}]]".format(coeff_string,self.matrix[0], self.matrix[1], self.matrix[2], self.matrix[3])
    
    def __mul__(self, rhs):
        return QuantumMatrix(self.matrix * rhs.matrix, self.coefficient * rhs.coefficient)
    
    def __add__(self, rhs):
        temp_rhs_matrix = sympy.Matrix([[1,0],[0,1]])
        for i,j in itertools.product([0,1], [0,1]):
                 temp_rhs_matrix[i,j] = sympy.simplify((rhs.matrix[i,j] * self.coefficient) / rhs.coefficient)
        return QuantumMatrix(self.matrix + temp_rhs_matrix, self.coefficient * 1/sympy.sqrt(2))
    
    def __sub__(self, rhs):
        return self + QuantumMatrix(rhs.matrix, rhs.coefficient * -1)
    
    def __eq__(self, rhs):
        return (self.matrix == rhs.matrix and self.coefficient == rhs.coefficient)
    
    def equiv(self, rhs):
        return (self.matrix == rhs.matrix)
    
    def __iter__(self):
        for x in self.matrix:
             yield x

**IonQ Single-Qubit Native Gates**

In [4]:
GPI = lambda phi : QuantumMatrix( [[0,sympy.exp(-sympy.I * phi)], 
                                   [sympy.exp(sympy.I * phi),0]])


GPI2 = lambda phi : QuantumMatrix( [[1, -sympy.I * sympy.exp(-sympy.I * phi)], 
                                     [ -sympy.I * sympy.exp(sympy.I * phi),1]], 1/sympy.sqrt(2))


**First, we define S and H gates using IonQ native gates. They will be the basis which we use to construct other Clifford gates** 

Hadamard Gates and S Gate

In [5]:
H = GPI(sympy.pi) * GPI2(sympy.pi/2)
display(sympy.Matrix(H.matrix))
print('Coefficient of H: ') 
H.coefficient

⎡1  1 ⎤
⎢     ⎥
⎣1  -1⎦

Coefficient of H: 


In [6]:
S =  GPI(sympy.pi/4) * GPI(sympy.pi)
display(sympy.Matrix(S.matrix))
print('Coefficient of S: ') 
S.coefficient

⎡1  0⎤
⎢    ⎥
⎣0  ⅈ⎦

Coefficient of S: 


In [7]:
Sdg = GPI(-sympy.pi/4) * GPI(sympy.pi)
display(sympy.Matrix(Sdg.matrix))
print('Coefficient of Sdg: ') 
Sdg.coefficient

⎡1  0 ⎤
⎢     ⎥
⎣0  -ⅈ⎦

Coefficient of Sdg: 


Pauli Groups

In [8]:
X =  GPI(sympy.pi)
display(sympy.Matrix(X.matrix))
print('Coefficient of X: ')  
X.coefficient

⎡0  1⎤
⎢    ⎥
⎣1  0⎦

Coefficient of X: 


In [9]:
Y =  GPI(sympy.pi/2)
display(sympy.Matrix(Y.matrix))
print('Coefficient of Y: ') 
Y.coefficient

⎡0   1⎤
⎢     ⎥
⎣-1  0⎦

Coefficient of Y: 


In [10]:
Z =  S * S
display(sympy.Matrix(Z.matrix))
print('Coefficient of Z: ') 
Z.coefficient

⎡1  0 ⎤
⎢     ⎥
⎣0  -1⎦

Coefficient of Z: 


In [11]:
I =  S * Sdg
display(sympy.Matrix(I.matrix))
print('Coefficient of I: ') 
I.coefficient

⎡1  0⎤
⎢    ⎥
⎣0  1⎦

Coefficient of I: 


The Axis Swap Group

In [12]:
# V = Sdg H
V =  GPI(-sympy.pi/4) * GPI(sympy.pi) * GPI(sympy.pi) * GPI2(sympy.pi/2)
display(sympy.Matrix(V.matrix))
print('Coefficient of V: ') 
V.coefficient

⎡1   1⎤
⎢     ⎥
⎣-ⅈ  ⅈ⎦

Coefficient of V: 


In [13]:
# W = H S
W =  GPI(sympy.pi) * GPI2(sympy.pi/2) * GPI(sympy.pi/4) * GPI(sympy.pi)
display(sympy.Matrix(W.matrix))
print('Coefficient of W: ') 
S.coefficient

⎡1  ⅈ ⎤
⎢     ⎥
⎣1  -ⅈ⎦

Coefficient of W: 


**24 Elements in Clifford Group**

Since the Clifford group is defined as the group of unitaries that normalize the Pauli group, we can write the one-qubit elements as $C_1 = AB$, where $A \in \{ I, V, W, H, HV, HW\}$ and $ B \in \{ I, X, Y, Z\}$ 

In [14]:
Clifford_Gates = {}

In [15]:
# X   =  GPI(sympy.pi)
# Y   =  GPI(sympy.pi/2)
# Z   =  GZ(sympy.pi/2)
# I   =  GZ(0)
# H   =  GPI(sympy.pi) * GPI2(sympy.pi/2)
# S   =  GPI(sympy.pi/4) * GPI(sympy.pi)
# Sdg =  GPI(-sympy.pi/4) * GPI(sympy.pi)
# V   =  GPI(-sympy.pi/4) * GPI(sympy.pi) * GPI(sympy.pi) * GPI2(sympy.pi/2)
# W   =  GPI(sympy.pi) * GPI2(sympy.pi/2) * GPI(sympy.pi/4) * GPI(sympy.pi)

In [16]:
# When A = I

Clifford_Gates['I'] = I
print ("I=")
display(sympy.Matrix(I.matrix))

Clifford_Gates['X'] = X
print ("X=")
display(sympy.Matrix(X.matrix))

Clifford_Gates['Y'] = Y
print ("Y=")
display(sympy.Matrix(Y.matrix))

Clifford_Gates['Z'] = Z
print ("Z=")
display(sympy.Matrix(Z.matrix))

I=


⎡1  0⎤
⎢    ⎥
⎣0  1⎦

X=


⎡0  1⎤
⎢    ⎥
⎣1  0⎦

Y=


⎡0   1⎤
⎢     ⎥
⎣-1  0⎦

Z=


⎡1  0 ⎤
⎢     ⎥
⎣0  -1⎦

In [17]:
# When A = V

# For simplicity, I will directly use V and XYZ gates, but the transformation to H ans S gates are below:
# V = H * S * H * S
# X = H * S * S * H
# Y = S * X * Sdg
# Z = S * S

Clifford_Gates['V'] = V
print ("V=")
display(sympy.Matrix(V.matrix))

VX = V * X
Clifford_Gates['VX'] = VX
print ("VX=")
display(sympy.Matrix(VX.matrix))

VY = V * Y
Clifford_Gates['VY'] = VY
print ("VY=")
display(sympy.Matrix(VY.matrix))

VZ = V * Z
Clifford_Gates['VZ'] = VZ
print ("VZ=")
display(sympy.Matrix(VZ.matrix))

V=


⎡1   1⎤
⎢     ⎥
⎣-ⅈ  ⅈ⎦

VX=


⎡1  1 ⎤
⎢     ⎥
⎣ⅈ  -ⅈ⎦

VY=


⎡1  -1⎤
⎢     ⎥
⎣ⅈ  ⅈ ⎦

VZ=


⎡1   -1⎤
⎢      ⎥
⎣-ⅈ  -ⅈ⎦

In [18]:
# When A = W

# For simplicity, I will directly use W and XYZ gates, but the transformation to H ans S gates are below:
# W = S * Z * H * S * Z * H
# X = H * S * S * H
# Y = S * X * Sdg
# Z = S * S

Clifford_Gates['W'] = W
print ("W=")
display(sympy.Matrix(W.matrix))

WX = W * X
Clifford_Gates['WX'] = WX
print ("WX=")
display(sympy.Matrix(WX.matrix))

WY = W * Y
Clifford_Gates['WY'] = WY
print ("WY=")
display(sympy.Matrix(WY.matrix))

WZ = W * Z
Clifford_Gates['WZ'] = WZ
print ("WZ=")
display(sympy.Matrix(WZ.matrix))

W=


⎡1  ⅈ ⎤
⎢     ⎥
⎣1  -ⅈ⎦

WX=


⎡1   -ⅈ⎤
⎢      ⎥
⎣-1  -ⅈ⎦

WY=


⎡1   ⅈ⎤
⎢     ⎥
⎣-1  ⅈ⎦

WZ=


⎡1  -ⅈ⎤
⎢     ⎥
⎣1  ⅈ ⎦

In [19]:
# When A = H

# For simplicity, I will directly use \ XYZ gates, but the transformation to H ans S gates are below:
# X = H * S * S * H
# Y = S * X * Sdg
# Z = S * S

Clifford_Gates['H'] = H
print ("H=")
display(sympy.Matrix(H.matrix))

HX = H * X
Clifford_Gates['HX'] = HX
print ("HX=")
display(sympy.Matrix(HX.matrix))

HY = H * Y
Clifford_Gates['HY'] = HY
print ("HY=")
display(sympy.Matrix(HY.matrix))

HZ = H * Z
Clifford_Gates['HZ'] = HZ
print ("HZ=")
display(sympy.Matrix(HZ.matrix))

H=


⎡1  1 ⎤
⎢     ⎥
⎣1  -1⎦

HX=


⎡1   1⎤
⎢     ⎥
⎣-1  1⎦

HY=


⎡1   -1⎤
⎢      ⎥
⎣-1  -1⎦

HZ=


⎡1  -1⎤
⎢     ⎥
⎣1  1 ⎦

In [20]:
# When A = HV

# For simplicity, I will directly use HV and XYZ gates, but the transformation to H ans S gates are below:
# W = S * Z * H * S * Z * H
# X = H * S * S * H
# Y = S * X * Sdg
# Z = S * S

HV = H * V
Clifford_Gates['HV'] = HV
print ("HV=")
display(sympy.Matrix(HV.matrix))

HVX = HV * X
Clifford_Gates['HVX'] = HVX
print ("HVX=")
display(sympy.Matrix(HVX.matrix))

HVY = HV * Y
Clifford_Gates['HVY'] = HVY
print ("HVY=")
display(sympy.Matrix(HVY.matrix))

HVZ = HV * Z
Clifford_Gates['HVZ'] = HVZ
print ("HVZ=")
display(sympy.Matrix(HVZ.matrix))

HV=


⎡1  ⅈ⎤
⎢    ⎥
⎣ⅈ  1⎦

HVX=


⎡1   -ⅈ⎤
⎢      ⎥
⎣-ⅈ  1 ⎦

HVY=


⎡1   ⅈ ⎤
⎢      ⎥
⎣-ⅈ  -1⎦

HVZ=


⎡1  -ⅈ⎤
⎢     ⎥
⎣ⅈ  -1⎦

In [21]:
# When A = HW

# For simplicity, I will directly use W and XYZ gates, but the transformation to H ans S gates are below:
# HW = H * S * H * S
# X = H * S * S * H
# Y = S * X * Sdg
# Z = S * S

HW = H * W
Clifford_Gates['HW'] = HW
print ("HW=")
display(sympy.Matrix(HW.matrix))

HWX = HW * X
Clifford_Gates['HWX'] = HWX
print ("HWX=")
display(sympy.Matrix(HWX.matrix))

HWY = HW * Y
Clifford_Gates['HWY'] = HWY
print ("HWY=")
display(sympy.Matrix(HWY.matrix))

HWZ = HW * Z
Clifford_Gates['HWZ'] = HWZ
print ("HWZ=")
display(sympy.Matrix(HWZ.matrix))

HW=


⎡1  0⎤
⎢    ⎥
⎣0  ⅈ⎦

HWX=


⎡0  1⎤
⎢    ⎥
⎣ⅈ  0⎦

HWY=


⎡0   1⎤
⎢     ⎥
⎣-ⅈ  0⎦

HWZ=


⎡1  0 ⎤
⎢     ⎥
⎣0  -ⅈ⎦

In [25]:
len(Clifford_Gates)

In [29]:
L = ['X','Y','Z']

In [ ]:
# X   =  GPI(sympy.pi)
# Y   =  GPI(sympy.pi/2)
# Z   =  SS = GPI(sympy.pi/4) * GPI(sympy.pi) * GPI(sympy.pi/4) * GPI(sympy.pi)
# I   =  SS+ = GPI(sympy.pi/4) * GPI(sympy.pi) * GPI(-sympy.pi/4) * GPI(sympy.pi)
# H   =  GPI(sympy.pi) * GPI2(sympy.pi/2)
# S   =  GPI(sympy.pi/4) * GPI(sympy.pi)
# Sdg =  GPI(-sympy.pi/4) * GPI(sympy.pi)
# V   =  GPI(-sympy.pi/4) * GPI(sympy.pi) * GPI(sympy.pi) * GPI2(sympy.pi/2)
# W   =  GPI(sympy.pi) * GPI2(sympy.pi/2) * GPI(sympy.pi/4) * GPI(sympy.pi)

In [92]:
def constructGate(qc, gate):
    if (gate == 'X'):
        qc.append(GPIGate(np.pi), [0])
    elif (gate == 'Y'):
        qc.append(GPIGate(np.pi/2), [0])
    elif (gate == 'Z'):
        qc.append(GPIGate(np.pi/4), [0])
        qc.append(GPIGate(np.pi), [0])
        qc.append(GPIGate(np.pi/4), [0])
        qc.append(GPIGate(np.pi), [0])
    elif (gate == 'I'):
        qc.append(GPIGate(np.pi/4), [0])
        qc.append(GPIGate(np.pi), [0])
        qc.append(GPIGate(-np.pi/4), [0])
        qc.append(GPIGate(np.pi), [0])
    elif (gate == 'H'):
        qc.append(GPIGate(np.pi), [0])
        qc.append(GPI2Gate(np.pi/2), [0])
    elif (gate == 'V'):
        qc.append(GPIGate(np.pi/4), [0])
        qc.append(GPIGate(np.pi), [0])
        qc.append(GPIGate(np.pi/4), [0])
        qc.append(GPIGate(np.pi), [0])
    return qc

In [93]:
def constructCircuit(L):
    qc = QuantumCircuit(1, name=f"gate-native")
    
    for i in range(len(L)):
        clifford_gate = split(L[i])
        for j in range(len(clifford_gate)):
            qc = constructGate(qc, clifford_gate[j])
        qc.draw()
    
    return qc

In [94]:
def split(word):
    return [char for char in word]

In [90]:
circuit = constructCircuit(L)

In [91]:
circuit.draw()

┌────────┐┌──────────┐┌──────────┐┌────────┐┌──────────┐┌────────┐
q: ┤ Gpi(π) ├┤ Gpi(π/2) ├┤ Gpi(π/4) ├┤ Gpi(π) ├┤ Gpi(π/4) ├┤ Gpi(π) ├
   └────────┘└──────────┘└──────────┘└────────┘└──────────┘└────────┘

In [ ]:
def get_inverse_matrix(clifford_string_mat):
    A = QuantumMatrix( [[1,0],
                        [0,1]])
    for i in range(len(clifford_string_mat)):
        A*=clifford_string_mat[i]
    return A
print(get_inverse_matrix(cliff_string_mat))

In [ ]:

def get_random_clifford_string(L,gates):
    entry_list = list(gates.items())
    cliff_string_name = []
    cliff_string_mat = []
    for i in range(L):
        random_choice = random.choice(entry_list)
        cliff_string_name.append(random_choice[0])
        cliff_string_mat.append(random_choice[1])
    return cliff_string_name, cliff_string_mat